# 1.1 🚀 Initialization and Configuration

* In this section, necessary libraries are imported, and the Semantic Kernel is configured with Azure OpenAI credentials loaded from a configuration file.

In [ ]:
#!import config/Settings.cs

In [ ]:
// Import Semantic Kernel
#r "nuget: Microsoft.SemanticKernel, 1.0.0-beta8"

In [ ]:
using Microsoft.SemanticKernel;

var builder = new KernelBuilder();

In [ ]:
using Kernel = Microsoft.SemanticKernel.Kernel;

var (model1, azureEndpoint1, apiKey1, gpt35TurboServiceId) = Settings.LoadFromFile(model: "gpt-35-turbo");
builder.WithAzureOpenAIChatCompletionService(model1, azureEndpoint1, apiKey1, serviceId: gpt35TurboServiceId);

var (model2, azureEndpoint2, apiKey2, gpt4ServiceId) = Settings.LoadFromFile(model: "gpt-4-32k");
builder.WithAzureOpenAIChatCompletionService(model2, azureEndpoint2, apiKey2, serviceId: gpt4ServiceId);

var kernel = builder.Build();

# 1.2 🚀 Semantic Function for Short Intent Extraction
* This section focuses on setting up the Semantic Function for extracting a concise user intent using a specific prompt.

In [ ]:
using Microsoft.SemanticKernel.TemplateEngine;
using Microsoft.SemanticKernel.TemplateEngine.Basic;
using Microsoft.SemanticKernel.Orchestration;
using Microsoft.SemanticKernel.Connectors.AI.OpenAI;
using Microsoft.SemanticKernel.AI;

var aiRequestSettings = new AIRequestSettings 
{
    ExtensionData = new Dictionary<string, object> { { "api-version", "2023-03-15-preview" } },
    ServiceId = gpt35TurboServiceId
};

In [ ]:
var input = "I want to find top-10 books about world history";

In [ ]:
// Chat with the Bot
// Create a simple prompt for the semantic function

string skPrompt = @"ChatBot: How can I help you?
User: {{$input}}

---------------------------------------------

The intent of the user in 25 words or less: ";

var getShortIntentFunction  = kernel.CreateSemanticFunction(skPrompt, "fn", "pn", requestSettings: aiRequestSettings);

var intentResult = await kernel.RunAsync(input, getShortIntentFunction);

Console.WriteLine(intentResult);

# 1.3 🚀 Semantic Function for Conversational Chat

* This section defines the Semantic Function for engaging in a more extended conversation with the chatbot, initializing the context, and executing the function.

In [ ]:
skPrompt = @"
ChatBot can have a conversation with you about any topic.
It can give detailed answer or say 'I don't know' if it does not have an answer.

{{$history}}
User: {{$userInput}}
ChatBot:";

In [ ]:
// Chat with the Bot

var aiRequestSettings = new AIRequestSettings 
{
    ExtensionData = new Dictionary<string, object> { { "api-version", "2023-03-15-preview" } },
    ServiceId = gpt4ServiceId
};

var chatFunction = kernel.CreateSemanticFunction(skPrompt, "fn", "pn", requestSettings: aiRequestSettings);
var context = kernel.CreateNewContext();

var history = "";
context.Variables["history"] = history;

var userInput = "Hi, I'm looking for suggestions of best sellers about world history";
context.Variables["userInput"] = userInput;

var bot_answer = await chatFunction.InvokeAsync(context);

In [ ]:
// Update the history with the output and set this as the new input value for the next request

history += $"\nUser: {userInput}\nSuggestions: {bot_answer.GetValue<string>()}\n";
context.Variables.Update(history);

Console.WriteLine(context);

# 1.4 🚀 Interactive Chat Function

* In this final section, an interactive chat function is defined, and several interactions with the chatbot are simulated. The complete chat history and the final user input are then printed. *

In [ ]:
// Chatting more)

Func<string, Task> Chat = async (string input) => {
    // Save new message in the context variables
    context.Variables["userInput"] = input;

    // Process the user message and get an answer
    var answer = await chatFunction.InvokeAsync(context);

    // Append the new interaction to the chat history
    history += $"\nUser: {input}\nSuggestions: {answer.GetValue<string>()}\n"; 
    context.Variables["history"] = history;
    
    // Show the response
    Console.WriteLine(context);
};

In [ ]:
await Chat("I would like a non-fiction book suggestion about Greece history. Please only list one book.");

In [ ]:
await Chat("that sounds interesting, what are some of the topics I will learn about?");

In [ ]:
await Chat("Which topic from the ones you listed do you think the most popular?");

In [ ]:
Console.WriteLine(context.Variables["history"]);

In [ ]:
Console.WriteLine(context.Variables["userInput"]);